In [100]:
import yfinance as yf
import pandas as pd
import datetime as dt
import numpy as np

In [101]:
stocks = ["TCS.NS"]
start_date = dt.datetime.now() - dt.timedelta(days=365)
end_date = dt.datetime.now()
data = {}
for stock in stocks:
    data[stock] = yf.download(stock, start=start_date, end=end_date, interval="1d")

[*********************100%***********************]  1 of 1 completed


## Compounded Annual Growth Rate (CAGR)

In [102]:
def CAGR(df):
    df = df.copy()
    df['return'] = df['Close'].pct_change()
    df['cum_return'] = (1 + df['return']).cumprod()
    n= len(df) / 252  # Assuming 252 trading days in a year
    cagr = (df['cum_return'].iloc[-1]) ** (1/n) - 1
    return cagr

In [103]:
for ticker in data:
    print("CAGR for {}: {}".format(ticker, CAGR(data[ticker])))

CAGR for TCS.NS: -0.25230078793652044


## Volaitility 

In [104]:
def volatility(df):
    df = df.copy()
    df['return'] = df['Close'].pct_change()
    vol = np.std(df['return']) * np.sqrt(252)  # Annualized volatility
    return vol

In [105]:
for ticker in data:
    print("Voloatility for {}: {}".format(ticker, volatility(data[ticker])))

Voloatility for TCS.NS: 0.20573195152232368


## Sharpe Ratio

In [106]:
def Sharpe(df):
    df = df.copy()
    sharpe = (CAGR(df)-0.03) / volatility(df)
    return sharpe

In [107]:
for ticker in data:
    print("Sharpe Ratio for {}: {}".format(ticker, Sharpe(data[ticker])))

Sharpe Ratio for TCS.NS: -1.3721776605316867


## Sortino Ratio

In [114]:

def Sortino(df, rfr):
    df = df.copy()
    cagr = CAGR(df)
    df['return'] = df['Close'].pct_change()
    negative_return = np.where(df['return'] > 0,0,df['return'])
    negative_volume = pd.Series(negative_return[negative_return != 0]).std() * np.sqrt(252)  # Annualized downside volatility
    sortino = (cagr - rfr) / negative_volume
    return sortino

In [115]:
for ticker in data:
    print("Sortino Ratio for {}: {}".format(ticker, Sortino(data[ticker], 0.03)))

Sortino Ratio for TCS.NS: -2.0187783055568334


## Maximum Drawdown

In [116]:
def max_dd(df):
    df = df.copy()
    df['return'] = df['Close'].pct_change()
    df['cum_return'] = (1 + df['return']).cumprod()
    df['peak'] = df['cum_return'].cummax()
    df['drawdown'] = df['peak'] - df['cum_return']
    return (df['drawdown']/ df['peak']).max()


In [117]:
for ticker in data:
    print("Maximum Drawdown for {}: {}".format(ticker, max_dd(data[ticker])))

Maximum Drawdown for TCS.NS: 0.2962099102840837


## Calamar Ratio

In [118]:
def calamar(df):
    df = df.copy()
    return CAGR(df) / max_dd(df)

In [119]:
for ticker in data:
    print("Calamar Ratio for {}: {}".format(ticker, calamar(data[ticker])))

Calamar Ratio for TCS.NS: -0.851763493309688
